In [1]:
from db_comm import PortfolioDBManager, DATABASE_NAME
from datetime import datetime

In [2]:
pm = PortfolioDBManager(DATABASE_NAME)

Deposit cash

In [3]:
#pm.deposit_cash(200)

View portfolio

In [4]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,CASH,99.3112,1.000,99.311200
1,AAPL,0.0010,688.758,0.688758


Insufuffcient cash for transaction

In [5]:
pm.record_transaction(
    tx_type="BUY",
    ticker="AAPL",
    shares=1,
    actual_price=150,
)

❌ Insufficient cash balance to BUY 150 of AAPL. Current CASH: 99.3112


'Transaction Denied: Insufficient Cash'

Calculation Motor

In [6]:
from portfolio_test.motor import CalculationMotor

In [7]:
cm = CalculationMotor("AAPL")

In [8]:
cm.df

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Price_SEK
Date,,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.468262,135480400,0.0,0.0,688.757977
2020-01-03,74.287498,75.144997,74.125000,74.357498,71.763718,146322800,0.0,0.0,690.873736
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.335564,118387200,0.0,0.0,683.061770
2020-01-07,74.959999,75.224998,74.370003,74.597504,71.995369,108872000,0.0,0.0,697.127991
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.153488,132079200,0.0,0.0,690.897009
...,...,...,...,...,...,...,...,...,...
2025-12-09,278.160004,280.029999,276.920013,277.179993,277.179993,32193300,0.0,0.0,2586.888034
2025-12-10,277.750000,279.750000,276.440002,278.779999,278.779999,33038300,0.0,0.0,2583.075000
2025-12-11,279.100006,279.589996,273.809998,278.029999,278.029999,33248000,0.0,0.0,2595.630057


Test buying aapl stock

In [9]:
oldest_date = cm.df.index.min()
cm.df.loc[oldest_date, "Price_SEK"]

np.float64(688.757977294922)

In [10]:
cm.df.Price_SEK.loc[oldest_date]

np.float64(688.757977294922)

In [16]:
pm.record_transaction(
    tx_type="BUY",
    ticker="AAPL",
    shares=.001,
    actual_price=cm.df.Price_SEK.loc[oldest_date],
    tx_datetime=oldest_date,
    currency="USD",
)

   -> Cash balance adjusted by -0.6888
✅ Recorded BUY: 0.001 AAPL @ 688.758. Snapshot updated.


In [11]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,CASH,99.3112,1.000,99.311200
1,AAPL,0.0010,688.758,0.688758


In [12]:
pm.get_cash_balance()

99.3112

Build calculation motor for AAPL

In [13]:
import pandas as pd

for date in cm.df.index:
    # if price today is lower than yesterday, buy as much as possible
    if date != cm.df.index.min():
        price = cm.df.Price_SEK.loc[date] # price in SEK
        yesterday = date - pd.Timedelta(days=1)
        price_yesterday = cm.df.Price_SEK.loc[yesterday]
        if price < price_yesterday:
            cash_balance = pm.get_cash_balance()
            shares_to_buy = cash_balance // price
            if shares_to_buy > 0:
                pm.record_transaction(
                    tx_type="BUY",
                    ticker="AAPL",
                    shares=shares_to_buy,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )
        # if price today is higher than yesterday, sell all shares
        elif price > price_yesterday:
            portfolio = pm.get_portfolio_snapshot()
            if "AAPL" in portfolio:
                shares_to_sell = portfolio["AAPL"]["shares"]
                if shares_to_sell > 0:
                    pm.record_transaction(
                        tx_type="SELL",
                        ticker="AAPL",
                        shares=shares_to_sell,
                        actual_price=price,
                        tx_datetime=date,
                        currency="SEK",
                    )


KeyError: Timestamp('2020-01-05 00:00:00')